This produces (x4) **image** for a LR image using a fine-tuned Baseline SRGAN


---


Download this notebook, upload it on google colab and then execute


---


Note that it only works for image with number of pixels less than or equal to 76,800, due to OOM error. Hence, if you upload an image of size more than the specified size, it will resize the image and then upsample it


---

Upload the input LR **.jpg** image and weights, preferably in the '/content/' folder. You can find the weights in the 'SRGAN-Finetuned' folder of this repo.


---


It is advisable to connect runtime to standard GPU


# Importing Libraries


In [8]:
import timeit
import math
import cv2 
import os
import numpy as np
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img
from tensorflow.keras.layers import Add, BatchNormalization, Conv2D, Dense, Flatten, Input, PReLU, Lambda
from tensorflow.keras.models import Model
from PIL import Image 
import PIL 

# Defining function to load images

In [59]:
def load_image(path):
    return np.array(Image.open(path))

# Defining the SRGAN model 

In [10]:
def resolve(model, LR_batch):
    LR_batch = tf.cast(LR_batch, tf.float32)
    SR_batch = model(LR_batch)
    SR_batch = tf.clip_by_value(SR_batch, 0, 255)
    SR_batch = tf.round(SR_batch)
    SR_batch = tf.cast(SR_batch, tf.uint8)
    return SR_batch

def single_resolve(model, LR):
    return resolve(model, tf.expand_dims(LR, axis=0))[0]

def pixelshuffler(scale):
    return lambda X: tf.nn.depth_to_space(X, scale)

In [11]:
def normalize_01(X):
    return X / 255.0

def normalize_m11(X):
    return X / 127.5 - 1

def denormalize_m11(X):
    return (X + 1) * 127.5

In [12]:
def Resnetblock(X_input, num_filters, momentum=0.8):
    X = Conv2D(num_filters, kernel_size=3, padding='same')(X_input)
    X = BatchNormalization(momentum=momentum)(X)
    X = PReLU(shared_axes=[1, 2])(X)
    X = Conv2D(num_filters, kernel_size=3, padding='same')(X)
    X = BatchNormalization(momentum=momentum)(X)
    X = Add()([X_input, X])
    return X

def Upsample(X_input, num_filters):
    X = Conv2D(num_filters, kernel_size=3, padding='same')(X_input)
    X = Lambda(pixelshuffler(scale=2))(X)
    return PReLU(shared_axes=[1, 2])(X)

def Generator(num_filters=64, num_of_resnet_blocks=16):
    X_in = Input(shape=(None, None, 3))
    X = Lambda(normalize_01)(X_in)

    X = Conv2D(num_filters, kernel_size=9, padding='same')(X)
    X = X_1 = PReLU(shared_axes=[1, 2])(X)

    for _ in range(num_of_resnet_blocks):
        X = Resnetblock(X, num_filters)

    X = Conv2D(num_filters, kernel_size=3, padding='same')(X)
    X = BatchNormalization()(X)
    X = Add()([X_1, X])

    X = Upsample(X, num_filters * 4)
    X = Upsample(X, num_filters * 4)

    X = Conv2D(3, kernel_size=9, padding='same', activation='tanh')(X)
    X = Lambda(denormalize_m11)(X)

    return Model(X_in, X)

SRGAN = Generator

# Instantiating the generator
Add the path to the weights file as an argument(str) to the load_weights() function in the following hidden cell

In [13]:
SRGAN_model = SRGAN()
SRGAN_model.load_weights('/content/ganweight(after 26k epochs).h5') #copy the path of our .h5 weights file and past it here

# Defining the function to apply SRGAN on image


In [60]:
def img_resizer(orig_img_path, h, w): 

  orig_img = cv2.imread(orig_img_path)
  A = 76800
  if (h>w):
    r = (h/w)
    new_w = int(math.sqrt(A/r))
    new_h = int(math.sqrt(A*r))
  else :
    r = (w/h)
    new_h = int(math.sqrt(A/r))
    new_w = int(math.sqrt(A*r))
  
  print("resized height=", new_h, "and resized width=", new_w)

  out = cv2.resize(orig_img, (new_w,new_h))

  cv2.imwrite('/content/input_img_resized.jpg', out)

In [66]:
def SRGAN_on_img(img_path):

  input = cv2.imread(img_path)

  h = input.shape[0]
  w = input.shape[1]
  
  print("orig height=", h, "and orig width=", w)
  if (h*w > 76800):
    img_resizer(img_path, h, w)
    input = cv2.imread('/content/input_img_resized.jpg')

  sr = single_resolve(SRGAN_model, input)
  out_name = '/content/finetuneSRGANoutput.png' 
  img_pil = array_to_img(sr)
  img1 = save_img(out_name, img_pil)

# Apply the fine-tuned generator on your image
Add the path(str) to your custom image as argument to the SRGAN_on_img() function given in the below cell

The output image will be saved as '/content/finetuneSRGANoutput.png'  and you can find the input resized image (if any) as '/content/input_img_resized.jpg'

In [67]:
SRGAN_on_img("/content/low_resolution_img.jpg")

orig height= 841 and orig width= 1327
resized height= 220 and resized width= 348
